In [1]:
from pulp import *
import networkx as nx
from matplotlib import pyplot as plt 

class WeightedDirectedGraph:
    def __init__(self, num_vertices, source_vertex_id, sink_vertex_id):
        self.__n = num_vertices
        self.__src = source_vertex_id
        self.__sink = sink_vertex_id
        self.__incoming = {}
        self.__outgoing = {}
        self.__edges = [] 
        self.__flow_lp_vars = []
        self.__lp_model = LpProblem('Flow Problem', LpMaximize)
        
    
    def add_edge(self, u, v, cap):
        assert cap > 0.0, 'capacity must be positive'
        assert u != self.__sink, 'Adding an edge leaving sink - not allowed'
        assert v != self.__src, 'Adding an edge entering source - not allowed'
        # Assign a edge id to the edge we are going to add
        edge_id = len(self.__edges)
        # Append the source/dest/capacity
        self.__edges.append( (u,v,cap))
        # Create a new LP variable
        lp_var = LpVariable(f'x_{edge_id}', 0.0, cap) # Create a decision variable with bounds between 0.0 and capacity of edge.
        # Append it to the list of decision variables
        self.__flow_lp_vars.append(lp_var)
        # Add the edge as incoming to vertex v
        if v in self.__incoming:
            self.__incoming[v].append(edge_id)
        else:
            lst = [edge_id]
            self.__incoming[v] = lst
        # add the edge as outgoing to vertex u
        if u in self.__outgoing:
            self.__outgoing[u].append(edge_id)
        else:
            lst=[edge_id]
            self.__outgoing[u] = lst 
        
    # This is a useful routine that uses the networkx library to 
    # draw the graph. We provide the option to draw the graph with just edge capacities
    # or to draw the graph showing the flow through each edge.
        
        
    def solve_flow_problem(self):
        # Objective is simply sum of all outgoing variables at source
        # Add the objective to the problem
        s = self.__src
        if s not in self.__outgoing:
            print('Warning: no outgoing edges at source. Max flow is trivially 0 -- bailing out.')
        else:
            lst_of_edge_ids = self.__outgoing[s]
            ob_expr = sum([self.__flow_lp_vars[edge] for edge in lst_of_edge_ids])
            self.__lp_model += ob_expr 
        # Add flow balance constraints
        for i in range(self.__n):
            if i != self.__src and i!= self.__sink: 
                incoming_lst = self.__incoming[i] if i in self.__incoming else []
                outgoing_lst = self.__outgoing[i] if i in self.__outgoing else [] 
                total_incoming_flow = sum([self.__flow_lp_vars[edge] for edge in incoming_lst])
                total_outgoing_flow = sum([self.__flow_lp_vars[edge] for edge in outgoing_lst])
                self.__lp_model += total_incoming_flow == total_outgoing_flow
        # Solve the LP
        self.__lp_model.solve()
        # Print the solution
        m = len(self.__edges)
        for i in range(m):
            v = self.__flow_lp_vars[i]
            (x, y, _) = self.__edges[i]
            print(f'Edge: {x} -> {y} : flow is {v.varValue}')
        print(f'Total flow is {value(self.__lp_model.objective)}')

In [2]:
G = WeightedDirectedGraph(5, 0, 9)
G.add_edge(0,1,5)
G.add_edge(0,3,3)
G.add_edge(0,4,4)
G.add_edge(1,2,9)
G.add_edge(1,4,6)
G.add_edge(2,3,8)
G.add_edge(3,4,7)



supplies = [-55, 100, -25, 35, -40]

G.solve_flow_problem()


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/70/jz6cvvk97t30vggrppkktlzh0000gn/T/519a33d3baa24c06bfe2b6f7cbb5ec12-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/70/jz6cvvk97t30vggrppkktlzh0000gn/T/519a33d3baa24c06bfe2b6f7cbb5ec12-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 46 RHS
At line 55 BOUNDS
At line 73 ENDATA
Problem MODEL has 8 rows, 17 columns and 29 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 4 (-4) rows, 6 (-11) columns and 10 (-19) elements
0  Obj -0 Dual inf 1.999998 (2)
3  Obj 8
Optimal - objective value 8
After Postsolve, objective 8, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 8 - 3 iterations time 0.002, Presolve 0.00
Option for printingOptions

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
